In [17]:
from __future__ import print_function
import tensorflow.keras,sklearn
# suppress tensorflow compilation warnings
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import tensorflow.compat.v1 as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
seed=4
np.random.seed(seed) # fix random seed
tf.compat.v1.set_random_seed(seed)
import matplotlib.pyplot as plt
import cv2
import pickle
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import requests
import io
from PIL import Image, ImageTk
import tkinter as tk
from tkinter.filedialog import asksaveasfilename
from tkinter.filedialog import askopenfilename
from numpy import asarray
import time
import tkinter as tk
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import pathlib
print(pathlib.Path().absolute())

C:\Users\Warsh\ML_Notebooks_edited\mlqdproj\proj_notebooks


Load trained models to be used in the classification of doodles. 

In [13]:
#load model from folder
possible_models = ['warshenstein','anna_banana','chris_is_a_model','jiinnet']
workdir = pathlib.Path().absolute()
models=[]
for mod in possible_models:
    models.append(load_model(workdir+mod+'.h5'))

Tkinter functions for upload, drawing, and classification. Run once before the final cell.

In [14]:
#upload image for classification cell
#image directory
userdir='C:/Users/Warsh/ML_Notebooks_edited/mlqdproj/drawings/'

#upload and format file, then send to classifier
def openFile():
    global imagepath
    imagepath = askopenfilename(title="Select User Settings", initialdir=userdir)
    image=Image.open(imagepath)
    if image.size[0]>image.size[1]:
        image=image.crop((image.size[0]/2-image.size[1]/2,0,image.size[0]/2+image.size[1]/2,image.size[1]))
    elif image.size[0]>image.size[1]:
        image=image.crop((0,image.size[1]/2-image.size[0]/2,image.size[0],image.size[1]/2+image.size[0]/2))
    else:
        image=image
    basewidth = 512
    wpercent = (basewidth/float(image.size[0]))
    hsize = int((float(image.size[1])*float(wpercent)))
    image = image.resize((basewidth,hsize), Image.ANTIALIAS)
    img = asarray(image)
    rgb_weights = [0.2989, 0.5870, 0.1141]
    gs_img1 = np.dot(img[...,:3], rgb_weights)
    gs_img = abs(255-gs_img1)/255
    lowres = cv2.resize(gs_img, dsize=(28, 28), interpolation=cv2.INTER_LINEAR)
    root.imagine = ImageTk.PhotoImage(Image.fromarray(gs_img1))
    change_pic()
    classify(lowres)

#draw pic and send to classifier
def draworama():
    global drawing,mode
    drawing = False # true if mouse is pressed
    mode = False # if True, draw rectangle. Press 'm' to toggle to curve
    ix,iy = -1,-1
    brushsize=10
    ersize=20
    # mouse callback function
    def draw_circle(event,x,y,flags,param):
        global ix,iy,drawing,mode

        #this is eraser
        if event == cv2.EVENT_RBUTTONDOWN:
            mode = not(mode)
            ix,iy = x,y

        #and this is for drawing
        if event == cv2.EVENT_LBUTTONDOWN:
            drawing = True
            ix,iy = x,y

        elif event == cv2.EVENT_MOUSEMOVE:
            if drawing == True:
                if mode == True:
                    cv2.circle(img,(x,y),ersize,(255,255,255),-1)
                else:
                    cv2.circle(img,(x,y),brushsize,(0,0,0),-1)

        elif event == cv2.EVENT_LBUTTONUP:
            drawing = False
            if mode == True:
                cv2.circle(img,(x,y),ersize,(255,255,255),-1)
            else:
                cv2.circle(img,(x,y),brushsize,(0,0,0),-1)

    #create image and a window and bind the function to window
    img = np.zeros((512,512,3), np.uint8)
    cv2.namedWindow('draw something!')
    cv2.circle(img,(255,255),400,(255,255,255),-1)
    cv2.setMouseCallback('draw something!',draw_circle)
    while(1):
        cv2.imshow('draw something!',img)
        if cv2.waitKey(20) & 0xFF == 27:
            break

    #convert to usable
    rgb_weights = [0.2989, 0.5870, 0.1141]
    gs_img1 = np.dot(img[...,:3], rgb_weights)
    gs_img = abs(255-gs_img1)/255
    lowres = cv2.resize(gs_img, dsize=(28, 28), interpolation=cv2.INTER_LINEAR)
    root.imagine = ImageTk.PhotoImage(Image.fromarray(img))
    change_pic()
    classify(lowres)
    #close windows
    cv2.destroyAllWindows()

#quit
def quitorama():
    root.destroy()

#change the displayed pic
def change_pic():
    vlabel.configure(image=root.imagine)

#classify
def classify(img):
    lowres = cv2.resize(img, dsize=(28, 28), interpolation=cv2.INTER_LINEAR)
    lowres_rshp=lowres.reshape(1,28,28,1)
    #predict with the trained CNN
    yokay=['airplane', 'monalisa', 'dragon', 'giraffe', 'axe', 'banana', 'eiffeltower', 'snail', 'windmill', 'snowman']
    cat1s=[]
    cert1s=[]
    cat2s=[]
    cert2s=[]
    cat3s=[]
    cert3s=[]
    for model in models:
        class_cert=model.predict(lowres_rshp)
        cat1=class_cert.argmax(1)
        cat1s.append(cat1[0])
        cert1 = 100*class_cert[0,cat1[0]]
        cert1s.append(cert1)
        class_cert[0,cat1[0]]=0
        cat2=class_cert.argmax(1)
        cat2s.append(cat2[0])
        cert2 = 100*class_cert[0,cat2[0]]
        cert2s.append(cert2)
        class_cert[0,cat2[0]]=0
        cat3=class_cert.argmax(1)
        cat3s.append(cat3[0])
        cert3 = 100*class_cert[0,cat3[0]]
        cert3s.append(cert3)
    #cat=class_cert.argmax(1)
    #cert=100*model.predict(lowres_rshp).max()
    #print(cert)
    #cert=class_cert[cat]
    lbl0.config(text = possible_models[0]+' classified drawing as ' + yokay[cat1s[0]] + ' with %.2f%% certainty\n'%cert1s[0] + 
                'second guess: ' + yokay[cat2s[0]] + ' with %.2f%% certainty\n'%cert2s[0] +
               'third guess: ' + yokay[cat3s[0]] + ' with %.2f%% certainty\n\n'%cert3s[0] +
               possible_models[1]+' classified drawing as ' + yokay[cat1s[1]] + ' with %.2f%% certainty\n'%cert1s[1] + 
                'second guess: ' + yokay[cat2s[1]] + ' with %.2f%% certainty\n'%cert2s[1] +
               'third guess: ' + yokay[cat3s[1]] + ' with %.2f%% certainty\n\n'%cert3s[1] +
               possible_models[2]+' classified drawing as ' + yokay[cat1s[2]] + ' with %.2f%% certainty\n'%cert1s[2] + 
                'second guess: ' + yokay[cat2s[2]] + ' with %.2f%% certainty\n'%cert2s[2] +
               'third guess: ' + yokay[cat3s[2]] + ' with %.2f%% certainty\n\n'%cert3s[2] +
               possible_models[3]+' classified drawing as ' + yokay[cat1s[3]] + ' with %.2f%% certainty\n'%cert1s[3] + 
                'second guess: ' + yokay[cat2s[3]] + ' with %.2f%% certainty\n'%cert2s[3] +
               'third guess: ' + yokay[cat3s[3]] + ' with %.2f%% certainty\n\n'%cert3s[3])# + possible_models[4]+' classified drawing as ' + yokay[cat1s[4]] + ' with %.2f%% certainty\n'%cert1s[4] + 'second guess: ' + yokay[cat2s[4]] + ' with %.2f%% certainty\n'%cert2s[4] + 'third guess: ' + yokay[cat3s[4]] + ' with %.2f%% certainty\n\n'%cert3s[4] + possible_models[5]+' classified drawing as ' + yokay[cat1s[5]] + ' with %.2f%% certainty\n'%cert1s[5] + 'second guess: ' + yokay[cat2s[5]] + ' with %.2f%% certainty\n'%cert2s[5] +'third guess: ' + yokay[cat3s[5]] + ' with %.2f%% certainty\n\n'%cert3s[5])

## Program Cell
Run this in order to use the pictionary program. You will have the option to upload or draw a doodle. If you choose to upload, it will classify this image on the right. If you draw, another window will open, left click is to draw, right click changes to and from an eraser, esc button will exit the doodle and classify it.

In [15]:
root = tk.Tk()
root.geometry("900x650")
frame1 = tk.Frame(root)
frame1.grid(row=0, column=0)

root.title("Pictionary")

url_main = 'https://physics.bu.edu/~warsh/ML_QD/'
response = requests.get(url_main + 'introslide.png')
response.raise_for_status()
photo = io.BytesIO(response.content)
#photo = 'C:/Users/Warsh/ML_Notebooks_edited/mlqdproj/drawings/introslide.png'
im = Image.open(photo)
ph = ImageTk.PhotoImage(im)


lbl0 = tk.Label(root, text="classify your drawing or image")
lbl0.grid(row=0, column=1)

vlabel=tk.Label(root,image=ph)
vlabel.image=ph
vlabel.grid(row=0, column=0)

b2=tk.Button(root,text="upload",height = 2, 
          width = 20,command=openFile)
b2.grid(row=1, column=0)
b3=tk.Button(root,text="draw",height = 2, 
          width = 20,command=draworama)
b3.grid(row=2, column=0)
q=tk.Button(root,text="quit",height = 2, 
          width = 20,command=quitorama)
q.grid(row=3, column=0)

root.mainloop()